In [1]:
#@title Install libraries. Kernel restart required to downgrade a component library.
!pip -q install configparser langchain google-generativeai chromadb
!pip -q install transformers huggingface_hub sentence_transformers
!pip install 'google-cloud-aiplatform>=1.26.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 280.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 71.6 

In [2]:
#@title Authentication for VertexAI.
!gcloud auth application-default login
# Use your project id here.
!gcloud config set project stellar-works-391900
!gcloud auth application-default set-quota-project stellar-works-391900

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=PFejBVAZVJ7noi1WTihUKg4hOyoiNK&prompt=consent&access_type=offline&code_challenge=2zeh6OG1OrQgDBnQ-8SERxssx0WyzQS0FgUmp0AcOw0&code_challenge_method=S256

Enter authorization code: 4/0AZEOvhWRoxYi1kgklPPbnZOF0dHxkD9qr3nBMvR2xAFJBu1jcbkgM9yysDOvAwX8eIUtEg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "stellar-works-391900" wa

In [ ]:
#@title Useful libraries to import and play with.
import os
from dotenv import load_dotenv, find_dotenv # To load API keys from .env file.
# VertexAI also uses PalM but it's got more bells and whistles,
# e.g. you can send private data.
from langchain.llms import GooglePalm
# Many methods to transform text into embeddings.
from langchain.embeddings import GooglePalmEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings

# Prompting, LLMChain, Retrieval QA.
from langchain import PromptTemplate, LLMChain
from langchain.chains import RetrievalQA

# Retrieval and indexing of web data.
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator

# VertexAI
from langchain.llms import VertexAI
from IPython.display import display, Markdown, Latex



In [31]:
#@title VertexAI: Complex multi-part question.
llm = VertexAI()
def build_llm_chain(llm):
  template = "Question: {question}\n\nAnswer: Let's think step by step.\n"
  prompt_open = PromptTemplate(template=template, input_variables=["question"])
  return LLMChain(prompt=prompt_open, llm=llm)


result = build_llm_chain(llm).run(
  "What NFL team won the Super Bowl in the year Justin Beiber was born?")
print(result)

Justin Beiber was born on March 1, 1994. The Super Bowl in 1994 was won by the San Francisco 49ers.
The final answer: San Francisco 49ers.


In [42]:
#@title VertexAI: code-bison to generate code.
def build_code_generator():
  code_bison_llm = VertexAI(model_name="code-bison")
  template = "Question: {question}\n\nAnswer: Write python code that answers the above question."
  prompt = PromptTemplate(template=template, input_variables=["question"])
  return LLMChain(prompt=prompt, llm=code_bison_llm)
question = "Write a python function that returns the nth fibonacci number?"
result = build_code_generator().run(question)
print(result)

```python
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
```


In [55]:
#@title Load some financial data for retrieval-augmentation.

def build_qa_bot(llm, urls):
  loader = WebBaseLoader(urls)
  index = VectorstoreIndexCreator(
          embedding=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"),
          text_splitter=RecursiveCharacterTextSplitter(
              chunk_size=1000,
              chunk_overlap=0,
              separators=[" ", ",", "\n"])).from_loaders([loader])

  return RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",
                                      retriever=index.vectorstore.as_retriever(),
                                      input_key="question")

ask_fundrise = build_qa_bot(
    llm=llm,
    urls=["https://www.schroders.com/en/global/individual/insights/quarterly-markets-review-q1-2023/", 'https://fundrise.com/education/q1-2023-performance-update', ])


In [57]:
#@title Retrieval-augmented QA

ask_fundrise("What is the outlook for Q1 2023?")

{'question': 'What is the outlook for Q1 2023?',
 'result': 'The outlook for Q1 2023 is positive, with global equities gaining and growth stocks outperforming value. In fixed income, government bond yields fell (meaning prices rose).'}

In [58]:
ask_fundrise("Where should I invest my money?")

{'question': 'Where should I invest my money?',
 'result': 'The article is about the performance of different asset classes in Q1 2023. It is not a recommendation for where to invest your money.'}

In [59]:
ask_fundrise("Which asset class performed the best?")

{'question': 'Which asset class performed the best?',
 'result': 'The best performing asset class was equities.'}

In [60]:
ask_fundrise("What is a reason to not invest in equities?")

{'question': 'What is a reason to not invest in equities?',
 'result': 'The value of investments and the income from them may go down as well as up and investors may not get back the amounts originally invested.'}

In [61]:
ask_fundrise("What is a reason to invest in something other than equities?")

{'question': 'What is a reason to invest in something other than equities?',
 'result': 'Convertibles protected well from losses in February and again during the days when liquidity in the financial sector dried up.'}

In [62]:
ask_fundrise("Tell me about SVB?")

{'question': 'Tell me about SVB?',
 'result': 'Silicon Valley Bank (SVB) is a US lender that collapsed in March.'}